<a href="https://colab.research.google.com/github/Atomix77/IMLO-IP/blob/main/IMLO_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import torch
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

Is CUDA supported by this system? True
CUDA version: 11.8


# Load Data



In [52]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

trainingTransform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(20),
      transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),])

testTransform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),])

trainingData = datasets.Flowers102(
    root = "./datasets",
    split = "train",
    transform = trainingTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

validationData = datasets.Flowers102(
    root = "./datasets",
    split = "val",
    transform = testTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

testData = datasets.Flowers102(
    root = "./datasets",
    split = "test",
    transform = testTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

# Dataloaders

In [53]:
batchSize = 64

trainingDataloader = DataLoader(trainingData, batchSize, shuffle = True)
validationDataloader = DataLoader(validationData, batchSize)
testDataloader = DataLoader(testData, batchSize)

# Get device

In [54]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Create Aritecture

In [55]:
class NeuralNetwork(nn.Module):
  def __init__(self, classAmount):
    super(NeuralNetwork, self).__init__()
    self.convStack =  nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.classifier = nn.Linear(128 * 28 * 28, classAmount)

  def forward(self, x):
    x = self.convStack(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

model = NeuralNetwork(102).to(device)

# Testing

In [56]:
def testModel(dataloader, model, lossFunction):
  print('Testing:')
  model.eval()
  total = 0
  correct = 0
  testLoss = 0
  
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      total += y.size(0)
      testLoss += lossFunction(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
  testLoss = testLoss/total
  correct = correct/total
  print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")

# Training

In [57]:
earlyStopCount = 3
def trainingModel(dataloader, model, lossFunction, optimizer, epochs):
  bestValLoss = float('inf')
  bestValAccuracy = 0
  waitCounter = 0

  for epoch in range(epochs):
    model.train()
    print(f'Epoch: {epoch+1}:')
    print('Training:')
    currentLoss = 0.0
    correct = 0
    total = 0
    testLoss = 0
    
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      optimizer.zero_grad()
      
      pred = model(X)
      loss = lossFunction(pred, y)

      loss.backward()
      optimizer.step()

      currentLoss += loss.item()

      total += y.size(0)
      testLoss += lossFunction(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    testLoss = testLoss/total
    correct = correct/total

    valLoss, valAccuracy = testModel(validationDataloader, model, lossFunction)
    print(f'Accuracy {(100*correct):>0.1f}%, Loss: {currentLoss / len(dataloader):.5f}, Test Losss: {testLoss:.5f}')
    
    if valLoss < bestValLoss:
      bestValLoss = valLoss
      waitCounter = 0
    else:
      waitCounter = 1

    if valAccuracy > bestValAccuracy:
      bestValAccuracy = valAccuracy

    if waitCounter >= earlyStopCount:
       print(f'Validation has not improved for {earlyStopCount} epochs.')
       break
  print(f'Best Accuracy: {bestValAccuracy}. Best Loss: {bestValLoss}')

In [58]:
learningRate = 0.001
batchSize = 128
epochs = 20

lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

trainingModel(trainingDataloader, model, lossFunction, optimizer, epochs)
testModel(testDataloader, model, lossFunction)
print("Finished")

Epoch: 1:
Training:
Testing:
Accuracy: 3.5%, Avg loss: 0.069225 



TypeError: cannot unpack non-iterable NoneType object

Current Best: Accuracy: 25.2%, Avg Loss: 0.103483

# Saving the model

In [ ]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model, 'model.pth')

# Loading the model

In [ ]:
# model = models.vgg16()
# model.load_state_dict(torch.load('model.pth'))
# model.eval()